In [6]:
%run modeling_THESIS.ipynb
%run training_utils_THESIS.ipynb
%run DataLoader_THESIS.ipynb

In [2]:
import random
import torch
import os
import pickle

In [3]:

def main(seed, batch_size, data_folder_path, lead1, lead2,
         input_dimension, output_dimension, 
         hidden_dimmension, attention_heads, 
         encoder_number_of_layers, decoder_number_of_layers, dropout,clip,
         n_epochs, window, stride,init_token, n_iters,
         # presentation
         num_plots,
         ):
    
    # Fix randomness
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    # Load splits dictionary
    saved_files = os.listdir(data_folder_path)
    with open(data_folder_path+"splits.pkl", 'rb') as handle:
        splits = pickle.load(handle)

    # Create data generators
    train_generator = DataGenerator(lead1, lead2,                        # leads
                                    data_folder_path,                    # path to relevant folder
                                    batch_size,                          # batch size
                                    list_IDs=splits['train'],            # list of relevat IDs (each ID is given in a <file>_<person> format)
                                    shuffle = True                       # Whether to shuffle the list of IDs at the end of each epoch.
                                    )
    validation_generator = DataGenerator(lead1, lead2,                        # leads
                                    data_folder_path,                    # path to relevant folder
                                    batch_size,                          # batch size
                                    list_IDs=splits['validation'],            # list of relevat IDs (each ID is given in a <file>_<person> format)
                                    shuffle = True                       # Whether to shuffle the list of IDs at the end of each epoch.
                                    )
    test_generator = DataGenerator(lead1, lead2,                        # leads
                                    data_folder_path,                    # path to relevant folder
                                    batch_size,                          # batch size
                                    list_IDs=splits['test'],            # list of relevat IDs (each ID is given in a <file>_<person> format)
                                    shuffle = True                       # Whether to shuffle the list of IDs at the end of each epoch.
                                    )
    
    
    # create a model
    model = TransformerModel(input_dimension, output_dimension, 
                        hidden_dimmension, attention_heads, 
                        encoder_number_of_layers, decoder_number_of_layers, dropout).to(device)

    optimizer = optim.AdamW(model.parameters())
    criterion = nn.MSELoss()

    print(f'The model has {count_parameters(model):,} trainable parameters')
    print(model)

    train_and_validate(
        # modeling
        model, 
        optimizer, criterion, clip,
        # data
        train_generator, validation_generator,
        # training
        window, stride, init_token,device, n_epochs, n_iters, 
        # plot validation at the end of an epoch
        num_plots,
        test_samples = None,
        initial_best_valid_loss = float('inf'))
    
    


In [10]:
    
config = {
    # general
    'seed':123,
    'data_folder_path': '../../dissertation/upstream_data/',
    'lead1':"LI", 'lead2':"LII",
    # training
    'batch_size':32,
    'n_epochs':10,
    # architecture - to be changed later and pushed out towards tuning
    'init_token': 0,
    'input_dimension':1, 'output_dimension':1,  # these should remain constant
    'hidden_dimmension': 64,                               # d_model (int) – the number of expected features in the input (required)???
    'attention_heads':None,                         # number of attention heads, if None then d_model//64
    'encoder_number_of_layers':3,
    'decoder_number_of_layers':3,
    'dropout':0.1,
    'clip':1,
    'window':500, 
    'stride':0.1,
    'n_iters': None, # number of signals to train on. All if None
    'num_plots':5,

}


main(**config)

The model has 350,787 trainable parameters
TransformerModel(
  (encoder): Linear(in_features=1, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Linear(in_features=1, out_features=64, bias=True)
  (pos_decoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
          )
          (linear1): Linear(in_features=64, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        

Within epoch loss (training) 0.0126:  68%|██████▊   | 874/1291 [2:22:17<1:09:42, 10.03s/it] 

In [ ]:
print('s')